In [18]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

import warnings
warnings.filterwarnings('ignore')

2.4.0+cu121
False
12.1


In [4]:
import torch_geometric
print(torch_geometric.__version__)

2.5.3


## Cora Dataset

In [6]:
# importing the Planetoid dataset
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='.', name='Cora')
dataset

Processing...
Done!
/home/consumer/Desktop/advanced-gnn/gnn_venv/lib/python3.10/site-packages/torch_geometric/io/fs.py:215: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ret

Cora()

In [7]:
# Cora has only one graph we can store in a dedicated variable
data = dataset[0]
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [8]:
print(f"Dataset: {dataset}")
print("--------------------")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of nodes: {data.x.shape[0]}")
print(f"Number of edges: {data.edge_index.shape[1]}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

Dataset: Cora()
--------------------
Number of graphs: 1
Number of nodes: 2708
Number of edges: 10556
Number of features: 1433
Number of classes: 7


we can also get detailed information from `PyTorch Geometric`'s inbuilt functions

In [12]:
print(f"Graph:")
print(f"======")
print(f"Edges are directed: {data.is_directed()}")
print(f"Has self loops: {data.contains_self_loops()}")
print(f"Contains isolated nodes: {data.contains_isolated_nodes()}")
print(f"Contains only one connected component: {data.is_coalesced()}")
print(f"Number of training nodes: {data.train_mask.sum().item()}")
print(f"Number of validation nodes: {data.val_mask.sum().item()}")
print(f"Number of test nodes: {data.test_mask.sum().item()}")


Graph:
Edges are directed: False
Has self loops: False
Contains isolated nodes: False
Contains only one connected component: False
Number of training nodes: 140
Number of validation nodes: 500
Number of test nodes: 1000


/home/consumer/Desktop/advanced-gnn/gnn_venv/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)


## Facebook Page-Page dataset

in this dataset, each of the `22,470` nodes represents an official Facebook page. Pages are connected when there are mutual likes between them.

The Facebook Page-Page dataset is similar to the previous one: it’s a social network with a node classification task. However, there are three major differences with Cora:<br>
-The number of nodes is much higher (2,708 versus 22,470)<br>
-The dimensionality of the node features decreased dramatically (from 1,433 to 128)<br>
-The goal is to classify each node into four categories instead of seven (which is easier since there are fewer options)

In [14]:
from torch_geometric.datasets import FacebookPagePage

dataset_fb = FacebookPagePage(root='.')
dataset_fb

Processing...
Done!
/home/consumer/Desktop/advanced-gnn/gnn_venv/lib/python3.10/site-packages/torch_geometric/io/fs.py:215: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ret

FacebookPagePage()

In [15]:
data_fb = dataset_fb[0]

In [16]:
print(f"Dataset: {dataset_fb}")
print("--------------------")
print(f"Number of graphs: {len(dataset_fb)}")
print(f"Number of nodes: {data_fb.x.shape[0]}")
print(f"Number of edges: {data_fb.edge_index.shape[1]}")
print(f"Number of features: {dataset_fb.num_features}")
print(f"Number of classes: {dataset_fb.num_classes}")

Dataset: FacebookPagePage()
--------------------
Number of graphs: 1
Number of nodes: 22470
Number of edges: 342004
Number of features: 128
Number of classes: 4


In [19]:
print(f"Graph:")
print(f"======")
print(f"Edges are directed: {data_fb.is_directed()}")
print(f"Has self loops: {data_fb.contains_self_loops()}")
print(f"Contains isolated nodes: {data_fb.contains_isolated_nodes()}")

Graph:
Edges are directed: False
Has self loops: True
Contains isolated nodes: False


#### Unlike Cora, FacebookPagePage doesn't have training, evaluation and test masks by default. so we can arbitrarily create masks with the `range()` function.

In [20]:
data_fb.train_mask = range(18000)
data_fb.val_mask = range(18001, 20000)
data_fb.test_mask = range(20001, 22470)

# Classifying nodes with Vanilla Neural Networks

Unlike the Zachary's Karate Club dataset, these two datasets contain an additional information: `node features`.<br>
They provide additional information about the nodes in a graph, such as a user's age, gender or interests in a social network.<br><br>
In a vanilla NN, these embeddings are directly used in the model to perform downstream tasks such as node classification. That means, here we will consider the node features as a regular dataset, <u>without</u> taking into account the topology of the network.

In [23]:
import pandas as pd

df_x = pd.DataFrame(data.x.numpy())
df_x['label'] = pd.DataFrame(data.y)

In [24]:
df_x.head()

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


We can now train a simple Multi-Layer Perceptron (MLP) on `data.x` with the labels provided by `data.y`.

In [25]:
# importing the necessary libraries
import torch
import torch.nn.functional as F
from torch.nn import Linear

In [26]:
def accuracy(y_pred, y_true):
    """Calculate accuracy."""
    return torch.sum(y_pred == y_true) / len(y_true)

In [28]:
class MLP(torch.nn.Module):
    def __init__(self, dim_input, dim_hidden, dim_output):
        super().__init__()
        self.linear1 = Linear(dim_input, dim_hidden)
        self.linear2 = Linear(dim_hidden, dim_output)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)

        return F.log_softmax(x, dim = 1)
    
    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr = 0.01, weight_decay = 5e-4)

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                                  data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')

    @torch.no_grad()      
    def test(self, data):
        self.eval()
        out = self(data.x)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

### Training the MLP on the Cora dataset

In [29]:
# Create MLP model
mlp = MLP(dataset.num_features, 16, dataset.num_classes)
print(mlp)

# Train
mlp.fit(data, epochs=100)

# Test
acc = mlp.test(data)
print(f'\nMLP test accuracy: {acc*100:.2f}%')

MLP(
  (linear1): Linear(in_features=1433, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=7, bias=True)
)
Epoch   0 | Train Loss: 1.954 | Train Acc: 19.29% | Val Loss: 2.01 | Val Acc: 12.60%
Epoch  20 | Train Loss: 0.103 | Train Acc: 100.00% | Val Loss: 1.47 | Val Acc: 50.20%
Epoch  40 | Train Loss: 0.013 | Train Acc: 100.00% | Val Loss: 1.60 | Val Acc: 49.40%
Epoch  60 | Train Loss: 0.007 | Train Acc: 100.00% | Val Loss: 1.60 | Val Acc: 48.40%
Epoch  80 | Train Loss: 0.008 | Train Acc: 100.00% | Val Loss: 1.48 | Val Acc: 50.40%
Epoch 100 | Train Loss: 0.009 | Train Acc: 100.00% | Val Loss: 1.42 | Val Acc: 51.40%

MLP test accuracy: 51.80%


### Training the MLP on the FacebookPagePage dataset

In [30]:
# Create MLP model
mlp = MLP(dataset_fb.num_features, 16, dataset_fb.num_classes)
print(mlp)

# Train
mlp.fit(data_fb, epochs=100)

# Test
acc = mlp.test(data_fb)
print(f'\nMLP test accuracy: {acc*100:.2f}%')

MLP(
  (linear1): Linear(in_features=128, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=4, bias=True)
)
Epoch   0 | Train Loss: 1.378 | Train Acc: 26.84% | Val Loss: 1.38 | Val Acc: 27.41%
Epoch  20 | Train Loss: 0.642 | Train Acc: 74.93% | Val Loss: 0.67 | Val Acc: 72.84%
Epoch  40 | Train Loss: 0.568 | Train Acc: 77.53% | Val Loss: 0.61 | Val Acc: 75.54%
Epoch  60 | Train Loss: 0.542 | Train Acc: 78.37% | Val Loss: 0.61 | Val Acc: 75.69%
Epoch  80 | Train Loss: 0.525 | Train Acc: 79.16% | Val Loss: 0.60 | Val Acc: 75.94%
Epoch 100 | Train Loss: 0.512 | Train Acc: 79.83% | Val Loss: 0.61 | Val Acc: 75.74%

MLP test accuracy: 75.46%


# Now we will do the same thing but this time will do it in a graphical architecture, thus, giving birth to the Vanilla Graph Neural Network

In [ ]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_input, dim_output):
        super().__init__()
        self.linear = Linear(dim_input, dim_output, bias = False)

    def forward(self, x, adjancy):
        x = self.linear(x)
        x = torch.sparse.mm(adjancy, x)
        return x

Before we can create our vanilla GNN, we need to convert the edge index from our dataset (`data.edge_index`) in coordinate format to a dense adjacency matrix. We also need to include `self` loops; otherwise, the central nodes won't be taken into account in their own embeddings.

In [31]:
from torch_geometric.utils import to_dense_adj

adjacency = to_dense_adj(data.edge_index)[0]
adjacency += torch.eye(len(adjacency))
adjacency

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])

In [32]:
# continue from page 80 (103)